<h1><center>DS 200 - Lec7: Pandas Part 2</center></h1>


## Introduction:

In this notebook, you will achieve the following learning objectives:

1. Learn how to handle missing data in a DataFrame.
2. Learn how to get aggregated statistics based off of each categorical group.
3. Learn how to put two DataFrames together using concatenation, merging and joining. 

## Section 1: Handle Missing Data

Let's show a few convenient methods to deal with Missing Data in pandas:

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'A':[1,2,np.nan],
                  'B':[5,np.nan,np.nan],
                  'C':[1,2,3]})

# Show df
df

,A,B,C
0,1.0,5.0,1
1,2.0,NaN,2
2,NaN,NaN,3


### 1: Drop all the missing values along rows or columns

By default, drop all the rows that has any missing data. 

In [3]:
df.dropna()



,A,B,C
0,1.0,5.0,1


But we can customize it to drop columns that are missing data. 

In [4]:
df.dropna(axis = 1)



,C
0,1
1,2
2,3


Argument `thresh=N` requires that a row or column has at least N non-NaNs to survive.

In [5]:
df.dropna(thresh = 2, axis = 0)



,A,B,C
0,1.0,5.0,1
1,2.0,NaN,2


Note `dropna()` only returns a temporary view, it will not change the data unless `inplace=True` is specified.

In [6]:
# Show df again
df

,A,B,C
0,1.0,5.0,1
1,2.0,NaN,2
2,NaN,NaN,3


### 2: Fill in the missing values with some contents

There's a whole philosophy and way of thinking and statistical methods for filling in missing values appropriately. It really depends on the dataset.

Use a string `FILL VALUE` to fill all missing values.

In [7]:
df.fillna('Fill Value')



,A,B,C
0,1.0,5.0,1
1,2.0,Fill Value,2
2,Fill Value,Fill Value,3


Most of the time we just fill for a single column or multiple columns. For example, fill the missing values in column A with the mean of column A. 

In [8]:
df['A'].fillna(value = df['A'].mean())




0    1.0
1    2.0
2    1.5
Name: A, dtype: float64

## Section 2: Groupby

The groupby method allows you to group rows of data together and call aggregate functions to each group. E.g. divide data base off of 'gender' --> 'male' group or 'female' group, and get the average 'age' for each group.

In [9]:
# Stock tickers
# Input data in Python dict format
data = {'Company':['GOOG','GOOG','MSFT','MSFT','FB','FB'],
       'Person':['Sam','Charlie','Amy','Vanessa','Carl','Sarah'],
       'Sales':[200,120,340,124,243,350]}

In [10]:
# Create dataframe
stock = pd.DataFrame(data)
stock

,Company,Person,Sales
0,GOOG,Sam,200
1,GOOG,Charlie,120
2,MSFT,Amy,340
3,MSFT,Vanessa,124
4,FB,Carl,243
5,FB,Sarah,350


Now you can use the .groupby() method to group rows together based off of a column name. For instance let's group based off of `Company`. This will create a `DataFrameGroupBy` object:

In [11]:
by_comp = stock.groupby('Company')


by_comp

This object contains the data within each group:

In [12]:
for comp, data in by_comp:
    print("Company:", comp)
    print("Data in group:", data)
    print("\n")

Company: FB
Data in group:   Company Person  Sales
4      FB   Carl    243
5      FB  Sarah    350


Company: GOOG
Data in group:   Company   Person  Sales
0    GOOG      Sam    200
1    GOOG  Charlie    120


Company: MSFT
Data in group:   Company   Person  Sales
2    MSFT      Amy    340
3    MSFT  Vanessa    124




You can save this object as a new variable, and then call aggregate methods off the object:

In [13]:
by_comp.mean()



,Sales
Company,
FB,296.5
GOOG,160.0
MSFT,232.0


#### Question: Only `Sales` column is shown, but no `Person` column. Why?

The second way is to select the "interested" column `Sales` before applying aggregate method. 

In [14]:
stock.groupby('Company')['Sales'].mean()



Company
FB      296.5
GOOG    160.0
MSFT    232.0
Name: Sales, dtype: float64

More examples of aggregate methods:

In [15]:
by_comp.std()



,Sales
Company,
FB,75.660426
GOOG,56.568542
MSFT,152.735065


In [16]:
by_comp.min()



,Person,Sales
Company,,
FB,Carl,243
GOOG,Charlie,120
MSFT,Amy,124


In [17]:
by_comp.max()



,Person,Sales
Company,,
FB,Sarah,350
GOOG,Sam,200
MSFT,Vanessa,340


In [18]:
by_comp.count()



,Person,Sales
Company,,
FB,2,2
GOOG,2,2
MSFT,2,2


In [19]:
by_comp.describe()



Sales                                                        
        count   mean         std    min     25%    50%     75%    max
Company                                                              
FB        2.0  296.5   75.660426  243.0  269.75  296.5  323.25  350.0
GOOG      2.0  160.0   56.568542  120.0  140.00  160.0  180.00  200.0
MSFT      2.0  232.0  152.735065  124.0  178.00  232.0  286.00  340.0

In [20]:
by_comp.describe().transpose()



Company              FB        GOOG        MSFT
Sales count    2.000000    2.000000    2.000000
      mean   296.500000  160.000000  232.000000
      std     75.660426   56.568542  152.735065
      min    243.000000  120.000000  124.000000
      25%    269.750000  140.000000  178.000000
      50%    296.500000  160.000000  232.000000
      75%    323.250000  180.000000  286.000000
      max    350.000000  200.000000  340.000000

In [21]:
by_comp.describe().transpose()['GOOG']





Sales  count      2.000000
       mean     160.000000
       std       56.568542
       min      120.000000
       25%      140.000000
       50%      160.000000
       75%      180.000000
       max      200.000000
Name: GOOG, dtype: float64

#### Group Exercise: Groupby

Given the following DataFrame. Answer the questions. 

In [22]:
weather = pd.DataFrame(data = [['1/1/2017', 'new york', 32, 6, 'Rain'],
                          ['1/2/2017', 'new york', 36, 7, 'Sunny'],
                          ['1/3/2017', 'new york', 28, 12, 'Snow'],
                          ['1/4/2017', 'new york', 33, 7, 'Sunny'],
                          ['1/1/2017', 'mumbai', 90, 5, 'Sunny'],
                          ['1/2/2017', 'mumbai', 85, 12, 'Fog'],
                          ['1/3/2017', 'mumbai', 87, 15, 'Fog'],
                          ['1/4/2017', 'mumbai', 92, 5, 'Rain'],
                          ['1/1/2017', 'paris', 45, 20, 'Sunny'],
                          ['1/2/2017', 'paris', 50, 13, 'Cloudy'],
                          ['1/3/2017', 'paris', 54, 8, 'Cloudy'],
                          ['1/4/2017', 'paris', 42, 10, 'Cloudy']],
                 index =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
                 columns= ['day', 'city', 'temperature', 'windspeed', 'event'])

# Display the DataFrame
weather

,day,city,temperature,windspeed,event
0,1/1/2017,new york,32,6,Rain
1,1/2/2017,new york,36,7,Sunny
2,1/3/2017,new york,28,12,Snow
3,1/4/2017,new york,33,7,Sunny
4,1/1/2017,mumbai,90,5,Sunny
5,1/2/2017,mumbai,85,12,Fog
6,1/3/2017,mumbai,87,15,Fog
7,1/4/2017,mumbai,92,5,Rain
8,1/1/2017,paris,45,20,Sunny
9,1/2/2017,paris,50,13,Cloudy


#### With this dataset, answer the following questions:
* 1. What was the maximum temperature in each of these 3 cities?
* 2. Which weather event has the highest average temperature?

In [23]:
# Q1
weather.groupby('city').max()['temperature']



city
mumbai      92
new york    36
paris       54
Name: temperature, dtype: int64

In [24]:
# Q2
weather.groupby('event').mean()['temperature'].sort_values(ascending = False)



event
Fog       86.000000
Rain      62.000000
Sunny     51.000000
Cloudy    48.666667
Snow      28.000000
Name: temperature, dtype: float64

___
## Section 3: Concatenation, Merging, and Joining

There are 3 main ways of combining DataFrames together: Merging, Joining and Concatenating. In this section we will discuss 3 Pandas functions/methods `pd.concat()`, `pd.merge()` and `df1.join(df2)` with examples.

### 1. Concatenation

Concatenation basically glues together DataFrames. Keep in mind that dimensions should match along the axis you are concatenating on. You can use **pd.concat** and pass in a list of DataFrames to concatenate together:

__Given the example DataFrames__

In [25]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                    'B': ['B0', 'B1', 'B2'],
                    'C': ['C0', 'C1', 'C2'],
                    'D': ['D0', 'D1', 'D2']},
                    index=[0, 1, 2])

In [26]:
df2 = pd.DataFrame({'A': ['A3', 'A4', 'A5'],
                    'B': ['B3', 'B4', 'B5'],
                    'C': ['C3', 'C4', 'C5'],
                    'D': ['D3', 'D4', 'D5']},
                    index=[3, 4, 5]) 

In [27]:
df3 = pd.DataFrame({'A': ['A6', 'A7', 'A8', 'A9'],
                    'B': ['B6', 'B7', 'B8', 'B9'],
                    'C': ['C6', 'C7', 'C8', 'C9'],
                    'D': ['D6', 'D7', 'D8', 'D9']},
                    index=[6, 7, 8, 9])

In [28]:
print(df1)

print(df2)

print(df3)

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
    A   B   C   D
3  A3  B3  C3  D3
4  A4  B4  C4  D4
5  A5  B5  C5  D5
    A   B   C   D
6  A6  B6  C6  D6
7  A7  B7  C7  D7
8  A8  B8  C8  D8
9  A9  B9  C9  D9


Concatenate all three along index (axis = 0)

In [29]:
new_df = pd.concat([df1, df2, df3], axis = 0)


new_df

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


Concatenate all three along columns (axis = 1). By default, `pd.concat()` uses 'outter' join on the key, which means only the matching indices can be combined into a single record; non-matching indices will be dispatched to different records.

In [30]:
new_df = pd.concat([df1, df2, df3], axis = 1)

new_df

,A,B,C,D,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,A3,B3,C3,D3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,A4,B4,C4,D4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,A5,B5,C5,D5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A6,B6,C6,D6
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A7,B7,C7,D7
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A8,B8,C8,D8
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A9,B9,C9,D9


### 2. Merging

The **merge** function allows you to merge DataFrames together using a similar logic as joining SQL Tables together. 

Here are the different types of the JOINs in SQL:

+ (INNER) JOIN: Returns records that have matching values in both tables
+ LEFT (OUTER) JOIN: Returns all records from the left table, and the matched records from the right table
+ RIGHT (OUTER) JOIN: Returns all records from the right table, and the matched records from the left table
+ FULL (OUTER) JOIN: Returns all records when there is a match in either left or right table

<img src="https://dataschool.com/assets/images/how-to-teach-people-sql/sqlJoins/sqlJoins_7.png" alt="SQL JOIN" width="600"/>


Given the following example DataFrames

In [31]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})
   
right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                          'C': ['C0', 'C1', 'C2', 'C3'],
                          'D': ['D0', 'D1', 'D2', 'D3']})    

In [32]:
left

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [33]:
right

,key,C,D
0,K0,C0,D0
1,K1,C1,D1
2,K2,C2,D2
3,K3,C3,D3


Merge them with 'inner' method on a single 'key' column

In [34]:
pd.merge(left, right, how = 'inner', on = 'key')



,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


Now, let's show a more complicated example with more than one key. 

In [35]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                        'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3']})
    
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                         'C': ['C0', 'C1', 'C2', 'C3'],
                         'D': ['D0', 'D1', 'D2', 'D3']})

In [36]:
left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [37]:
right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


In [38]:


pd.merge(left, right, how = 'inner', on = ['key1', 'key2'])


,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


In [39]:
pd.merge(left, right, how = 'outer', on = ['key1', 'key2'])



,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


In [40]:
pd.merge(left, right, how = 'right', on = ['key1', 'key2'])




,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


In [41]:
pd.merge(left, right, how = 'left', on = ['key1', 'key2'])




,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN


#### Group exercise: merge

Given the DataFrames df1 and df2 in the following way:

In [42]:
df1 = pd.DataFrame({
    'city':['new york', 'chicago', 'orlando', 'baltimore'],
    'temperature':[21, 14, 35, 25],
    'humidity':[65, 68, 71, 75]
})
df1

,city,temperature,humidity
0,new york,21,65
1,chicago,14,68
2,orlando,35,71
3,baltimore,25,75


In [43]:
df2 = pd.DataFrame({
    'city': ['chicago', 'new york', 'san diego'],
    'temperature':[14, 21, 35],
    'humidity': [65, 68, 73]
})
df2

,city,temperature,humidity
0,chicago,14,65
1,new york,21,68
2,san diego,35,73


#### Q1: Write on a paper the result of merge df1 and df2 together in the `inner` way on key `city`. Then check it out with code.

In [44]:
inner = pd.merge(df1, df2, how = 'inner', on = 'city')



# Display inner
inner

,city,temperature_x,humidity_x,temperature_y,humidity_y
0,new york,21,65,21,68
1,chicago,14,68,14,65


#### Notice the column names look a bit weird. Why? Now let's try to improve the column names.

In [45]:
inner_col_name = pd.merge(df1, df2, how = 'inner', on = 'city', suffixes=('_left', '_right'), indicator = True)



inner_col_name

,city,temperature_left,humidity_left,temperature_right,humidity_right,_merge
0,new york,21,65,21,68,both
1,chicago,14,68,14,65,both


#### Q2: What if merging df1 and df2 in the `left` way on key `city` and `temperature`?

In [46]:
left = pd.merge(df1, df2, how = 'left', on = ['city', 'temperature'], suffixes=('_left', '_right'), indicator = True)



left

,city,temperature,humidity_left,humidity_right,_merge
0,new york,21,65,68.0,both
1,chicago,14,68,65.0,both
2,orlando,35,71,NaN,left_only
3,baltimore,25,75,NaN,left_only


### 3. Joining
Joining is a convenient method (NOT a function!) for combining the columns of two potentially differently-indexed DataFrames into a single result DataFrame. You don't need to specify 'key' for join, because it takes the indexes from the DataFrames as the KEY. But you still need to specify 'how'.

In [47]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                      index=['K0', 'K1', 'K2']) 

right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                    'D': ['D0', 'D2', 'D3']},
                      index=['K0', 'K2', 'K3'])

print("Left:", left, sep = '\n')
print("\nRight:", right, sep = '\n')

Left:
     A   B
K0  A0  B0
K1  A1  B1
K2  A2  B2

Right:
     C   D
K0  C0  D0
K2  C2  D2
K3  C3  D3


Different from merge() function, the default 'how' is 'left' join.

In [48]:
left.join(right)



,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


In [49]:
left.join(right, how = 'outer')



,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


In [54]:
right.join(left, how ='outer')



,C,D,A,B
K0,C0,D0,A0,B0
K1,NaN,NaN,A1,B1
K2,C2,D2,A2,B2
K3,C3,D3,NaN,NaN


# Great Job!